In [ ]:
from db import DB
from utils.pytorch import seed_everything, get_device
from models.pytorch.us_gaap_alignment import UsGaapAlignmentModel
from models.pytorch.us_gaap_alignment.inference import ingest_automated_inference


device = get_device()

db = DB()

# Load reference dataset
dataset_path = "data/us_gaap_concepts_with_variations_and_embeddings.jsonl"

model = UsGaapAlignmentModel.from_pretrained(
    "data/pretrained",
    device=device
)

In [ ]:
ingest_automated_inference(db, dataset_path, model, device)